In [1]:
import copy
import os
import argparse
import torch
import numpy as np
import random

%load_ext autoreload
%autoreload 2
from erasure.utils.config.local_ctx import Local
from erasure.utils.config.global_ctx import Global 
from erasure.core.factory_base import ConfigurableFactory
from erasure.data.datasets.DatasetManager import DatasetManager

In [4]:
config_file = os.path.join("configs", "example_celeba.jsonc")
#config_file = os.path.join("configs", "example.jsonc")


global_ctx = Global(config_file)
global_ctx.factory = ConfigurableFactory(global_ctx)

global_ctx.logger.info(f"Current PyTorch seed: {torch.initial_seed()}")

#Create Dataset
print("CONFIG DATA ", global_ctx.config.data)
data_manager = global_ctx.factory.get_object( Local( global_ctx.config.data ))

2025-01-04 23:34:12,863265051 | INFO | 2527015 - Creating Global Context for: configs/example_celeba.jsonc
2025-01-04 23:34:12,863265083 | INFO | 2527015 - WARNING - SEEDS ARE RANDOMLY GENERATED AS 1429575165 - Add globals['seed'] to the main Cfg to fix them.
2025-01-04 23:34:12,863265085 | INFO | 2527015 - Caching System: False.
2025-01-04 23:34:12,863265114 | INFO | 2527015 - Current PyTorch seed: 1429575165
CONFIG DATA  {'class': 'erasure.data.datasets.DatasetManager.DatasetManager', 'parameters': {'DataSource': {'class': 'erasure.data.data_sources.TVDataSource.TVDataSourceCelebA', 'parameters': {'path': 'torchvision.datasets.CelebA', 'transform': ['ToTensor'], 'label_column': 'attr', 'target_type': ['attr', 'identity'], 'preprocess': [{'class': 'erasure.data.preprocessing.reshape_y_z.reshape_y_z', 'parameters': {'keep_as_y': [0, 0], 'move_to_z': [1]}}]}}, 'partitions': [{'class': 'erasure.data.datasets.DataSplitter.DataSplitterPercentage', 'parameters': {'parts_names': ['all_reduce

Filtering Data:  33%|███▎      | 1/3 [03:19<06:39, 199.81s/it]


KeyboardInterrupt: 

In [3]:
data_manager.n_classes

2

In [4]:
loader, _ = data_manager.get_loader_for('forget')

In [6]:
i=0
for X,y in loader:
    print(len(y))
    i = i +1
    if i == 3:
        break

30


In [6]:
loader

In [7]:
loader, _ = data_manager.get_loader_for('train')

2024-12-11 16:38:20,-1235287196 | INFO | 1975738 - TRAINING WITH 700 samples


In [ ]:
i=0
for X,y in loader:
    print(X.shape)
    i = i +1
    if i == 3:
        break

torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])
torch.Size([128, 1, 28, 28])


In [17]:
data_manager.partitions['forget']
loader, _ = data_manager.get_loader_for('forget')
i=0
for X,y in loader:
    print(y)
    i = i +1
    if i == 3:
        break

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


In [20]:

from erasure.utils.config.global_ctx import bcolors


current = Local(global_ctx.config.predictor)
current.dataset = data_manager
predictor = global_ctx.factory.get_object(current)
global_ctx.logger.info('Global Predictor: ' + str(predictor))

#Create unlearners 
unlearners = []
unlearners_cfg = global_ctx.config.unlearners
for un in unlearners_cfg:
    current = Local(un)
    current.dataset = data_manager
    current.predictor = copy.deepcopy(predictor)
    unlearners.append( global_ctx.factory.get_object(current) )


#Evaluator
current = Local(global_ctx.config.evaluator)
current.unlearners = unlearners
evaluator = global_ctx.factory.get_object(current)

# Evaluations
for unlearner in unlearners:
    global_ctx.logger.info(f'''{bcolors.OKGREEN}####\t\t Evaluating: {unlearner.__class__.__name__} \t\t####{bcolors.ENDC}''')
    evaluator.evaluate(unlearner,predictor)

 

2024-12-11 15:37:35,-1238931755 | INFO | 1950243 - Instantiating: erasure.model.classifiers.standard_classifier.IrisNN
2024-12-11 15:37:35,-1238931752 | INFO | 1950243 - Instantiating: torch.optim.Adam
2024-12-11 15:37:35,-1238931751 | INFO | 1950243 - Instantiating: torch.nn.CrossEntropyLoss
2024-12-11 15:37:35,-1238931720 | INFO | 1950243 - TRAINING WITH 120 samples
2024-12-11 15:37:35,-1238931697 | INFO | 1950243 - epoch = 0 ---> loss = 2.0275	 accuracy = 0.4167
2024-12-11 15:37:35,-1238931673 | INFO | 1950243 - epoch = 1 ---> loss = 1.3810	 accuracy = 0.4167
2024-12-11 15:37:35,-1238931648 | INFO | 1950243 - epoch = 2 ---> loss = 0.9923	 accuracy = 0.4167
2024-12-11 15:37:35,-1238931604 | INFO | 1950243 - epoch = 3 ---> loss = 0.8879	 accuracy = 0.8333
2024-12-11 15:37:36,-1238931580 | INFO | 1950243 - epoch = 4 ---> loss = 0.8840	 accuracy = 0.4333
2024-12-11 15:37:36,-1238931538 | INFO | 1950243 - epoch = 5 ---> loss = 0.8328	 accuracy = 0.5083
2024-12-11 15:37:36,-1238931506 | I